In [20]:
import pandas as pd
import numpy as np
from random import shuffle
import os
import scipy
import sqlite3
from collections import defaultdict

In [2]:
curr_path = os.path.realpath('')
print(curr_path)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
cur = conn.cursor()
cur.execute("select * from loan limit 5;")
results = cur.fetchall()
# print(results)
cur.close()
conn.close()

C:\Users\andre\Documents\GitHub\loan_driving2


In [3]:
conn = sqlite3.connect("lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status = 'Charged Off';", conn)
print(df)
conn.close()

       loan_amnt        id  loan_status sub_grade
0         2500.0   1077430  Charged Off        C4
1         5600.0   1071795  Charged Off        F2
2         5375.0   1071570  Charged Off        B5
3         9000.0   1064687  Charged Off        C1
4        10000.0   1069057  Charged Off        B2
5        21000.0   1039153  Charged Off        B4
6         6000.0   1069559  Charged Off        B3
7        15000.0   1069800  Charged Off        C2
8         5000.0   1069657  Charged Off        D2
9         5000.0   1069465  Charged Off        A5
10       15000.0   1069248  Charged Off        B1
11       12000.0   1069243  Charged Off        C5
12       21000.0   1069410  Charged Off        E4
13       10000.0   1069126  Charged Off        B2
14       12500.0   1069093  Charged Off        D3
15        8200.0   1068906  Charged Off        F2
16        6400.0   1060981  Charged Off        D2
17       17500.0   1068542  Charged Off        D3
18        6200.0   1068509  Charged Off        B1


In [6]:
ddict = df.to_dict()
print(ddict['loan_amnt'][7])
ddict['sub_grade'][7]
g = zip(ddict['loan_amnt'].values(),ddict['sub_grade'].values(),ddict['id'].values(),ddict['loan_status'].values())
print(len(list(g)))

15000.0
45248


In [7]:
# create a training set
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select loan_amnt,id,loan_status,sub_grade from loan where loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)';", conn)
# print(df.axes)
# print(df.get_values()[0][1])
# print(df.get_values())
# print(df.values[3])
print(df.values[0])

conn.close()
# feature vector - predictor will predict ("charged off/default status") vs. 
# [1(maybe check this),'sub_grade','loan_amnt','emp_length','home_ownership','annual_inc','verification_status','dti'(debt-to-income),'']

[5000.0 '1077501' 'Fully Paid' 'B2']


In [8]:
# find all unique values for the loan status column
l_status_dict = defaultdict(int)
for row in df.itertuples():
    # print(row[3])
    if row[3] not in l_status_dict:
        l_status_dict[row[3]] += 1

# print(l_status_dict.keys())

In [9]:
print(l_status_dict.keys())

dict_keys(['Fully Paid', 'Charged Off', 'Default', 'In Grace Period', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Issued'])


In [10]:
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
# very important, select a random sample of the data for our training set & validation sets 
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
# df_validation = pd.read_sql_query("select * from loan where id in(selec id from loan order by random() limit 100000);", conn)


In [27]:
# split the data sets into training and validation
#for row in df.itertuples():
#   print(row[1])
# shuffle that dataframe
df = df.sample(frac=1).reset_index(drop=True)
data_train = df[:len(df)]
data_validation = df[len(df):]

In [37]:
# set up 

sub_g_labels = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3'\
            ,'D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']

# make sub_grade 
# give sub grade numeric value.
grade_v = [(1.0/35 * i) for i in range(1, 36)]

# dictionary to make sub_grade val to sub_grade string
sub_grade_dict = dict(zip(sub_g_labels, grade_v))

print(sub_grade_dict)
print(df.axes)

{'A1': 0.02857142857142857, 'A2': 0.05714285714285714, 'A3': 0.08571428571428572, 'A4': 0.11428571428571428, 'A5': 0.14285714285714285, 'B1': 0.17142857142857143, 'B2': 0.19999999999999998, 'B3': 0.22857142857142856, 'B4': 0.2571428571428571, 'B5': 0.2857142857142857, 'C1': 0.3142857142857143, 'C2': 0.34285714285714286, 'C3': 0.37142857142857144, 'C4': 0.39999999999999997, 'C5': 0.42857142857142855, 'D1': 0.45714285714285713, 'D2': 0.4857142857142857, 'D3': 0.5142857142857142, 'D4': 0.5428571428571428, 'D5': 0.5714285714285714, 'E1': 0.6, 'E2': 0.6285714285714286, 'E3': 0.6571428571428571, 'E4': 0.6857142857142857, 'E5': 0.7142857142857143, 'F1': 0.7428571428571429, 'F2': 0.7714285714285714, 'F3': 0.7999999999999999, 'F4': 0.8285714285714285, 'F5': 0.8571428571428571, 'G1': 0.8857142857142857, 'G2': 0.9142857142857143, 'G3': 0.9428571428571428, 'G4': 0.9714285714285714, 'G5': 1.0}
[RangeIndex(start=0, stop=200000, step=1), Index(['index', 'id', 'member_id', 'loan_amnt', 'funded_amnt',


In [30]:
# need to build my feature vecotr
# X_train = f['sub_grade','loan_amnt']
# need y predictions combine(charged off/default(True(1)) or fully paid(False(0)))
# y_train =[]

def feature(datum):
    # create our simplistic featur vector
    return [sub_grade_dict[datum['sub_grade']], loan_amount = datum['loan_amnt']]


def prediction_of_y(datum):
    if datum['loan_status'] == ('Charged Off' || 'Default'):
        return True
    else:
        return False

X_train = [feature(d) for d in data_train]
y_train = [prediction_of_y(d) for d in data_train]
    

{'A1': 0.02857142857142857, 'A2': 0.05714285714285714, 'A3': 0.08571428571428572, 'A4': 0.11428571428571428, 'A5': 0.14285714285714285, 'B1': 0.17142857142857143, 'B2': 0.19999999999999998, 'B3': 0.22857142857142856, 'B4': 0.2571428571428571, 'B5': 0.2857142857142857, 'C1': 0.3142857142857143, 'C2': 0.34285714285714286, 'C3': 0.37142857142857144, 'C4': 0.39999999999999997, 'C5': 0.42857142857142855, 'D1': 0.45714285714285713, 'D2': 0.4857142857142857, 'D3': 0.5142857142857142, 'D4': 0.5428571428571428, 'D5': 0.5714285714285714, 'E1': 0.6, 'E2': 0.6285714285714286, 'E3': 0.6571428571428571, 'E4': 0.6857142857142857, 'E5': 0.7142857142857143, 'F1': 0.7428571428571429, 'F2': 0.7714285714285714, 'F3': 0.7999999999999999, 'F4': 0.8285714285714285, 'F5': 0.8571428571428571, 'G1': 0.8857142857142857, 'G2': 0.9142857142857143, 'G3': 0.9428571428571428, 'G4': 0.9714285714285714, 'G5': 1.0}


In [36]:
print(df['loan_amnt'].max())
print(df['loan_amnt'].min())

35000.0
600.0
